This notebook implements an efficient version of pseudobulk nb-glm based differential expression analysis with DESeq2. Pseudobulk means that all reads from a single batch group (e.g. donor) get pooled into a single observation. 

In general, pseudobulk is statistically preferable to but much slower than Wilcoxon, especially when you need to consider covariates. A more robust but considerably slower alternative to pseudobulk is including donors as random effects. Random effects are preferable for small cell count groups but likely give similar results to pseudobulk estimates for large groups. 

This idea is not at all new. The earliest reference I know for is from Lun et al: 
https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0947-7. 


A few implementation notes: 

1) To find markers most upregulated in a cluster, I divide samples into those in and out of the cluster. An alternative is to let each out group remain an independent pseudobulk sample. This is in fact the recommended way from Mike Love: https://support.bioconductor.org/p/118090/. While this is certainly faster than re-estimate size factors for each cluster-specific analysis, I find it gives strange results. Namely, I get more inflated p-values and significant p-values for the wrong canonical marker genes (e.g. CD14 for B cells).  

2) On my laptop, it takes ~20 seconds to run do ~3000 genes from 2700 cells, 3 donors, 2 batches, and 9 cell types. 

# Load some data

In [6]:
devtools::load_all('..')

ℹ Loading presto



In [7]:
suppressPackageStartupMessages({
    library(tidyverse)
#     library(presto)
    library(singlecellmethods)
    library(SeuratData)
    library(Seurat)
    library(DESeq2)    
})

fig.size <- function (h, w) 
{
    options(repr.plot.height = h, repr.plot.width = w)
}

Load small dataset for exposition

In [8]:
if (!SeuratData::AvailableData()['pbmc3k.SeuratData', 'Installed']) {
    SeuratData::InstallData("pbmc3k")
}
data("pbmc3k")

Add fake donor and batch columns

In [9]:
pbmc3k@meta.data$donor <- factor(sample(LETTERS[1:3], ncol(pbmc3k), TRUE))
pbmc3k@meta.data$batch <- factor(sample(LETTERS[1:2], ncol(pbmc3k), TRUE))

In [10]:
head(pbmc3k@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,seurat_annotations,donor,batch
,<fct>,<dbl>,<int>,<fct>,<fct>,<fct>
AAACATACAACCAC,pbmc3k,2419,779,Memory CD4 T,A,B
AAACATTGAGCTAC,pbmc3k,4903,1352,B,A,B
AAACATTGATCAGC,pbmc3k,3147,1129,Memory CD4 T,A,A
AAACCGTGCTTCCG,pbmc3k,2639,960,CD14+ Mono,A,B
AAACCGTGTATGCG,pbmc3k,980,521,NK,A,B
AAACGCACTGGTAC,pbmc3k,2163,781,Memory CD4 T,B,A


# Test 

## Collapse to pseudobulk

In [11]:
data_collapsed <- collapse_counts(pbmc3k@assays$RNA@counts, 
                                  pbmc3k@meta.data, 
                                  c('seurat_annotations', 'donor', 'batch'))
head(data_collapsed$meta_data)

CAREFUL: get_norm makes very strong assumptions about data



,seurat_annotations,donor,batch,N,logUMI
,<fct>,<fct>,<fct>,<int>,<dbl>
sample_28,Memory CD4 T,A,B,88,12.36016
sample_30,B,A,B,56,11.67743
sample_1,Memory CD4 T,A,A,83,12.35118
sample_29,CD14+ Mono,A,B,82,12.23536
sample_33,NK,A,B,33,11.09052
sample_10,Memory CD4 T,B,A,90,12.39591


## Do DESeq2

In [12]:
res_mat <- pseudobulk_deseq2(~seurat_annotations + donor + batch, 
                             data_collapsed$meta_data,
                             data_collapsed$counts_mat, verbose = TRUE)


ERROR: Error in pseudobulk_deseq2(~seurat_annotations + donor + batch, data_collapsed$meta_data, : could not find function "pseudobulk_deseq2"


In [ ]:
head(res_mat)

In [ ]:
top_markers_dds(res_mat, lfc_min = 1, padj_max = .05)

## Volcano plots

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 8)
res_mat %>% 
    ggplot(aes(log2FoldChange, -log10(pvalue), color = padj < .01 & abs(log2FoldChange) > 1)) + 
        geom_point(shape = 21) + 
        facet_wrap(~group, scales = 'free') + 
        guides(color = FALSE) + 
        NULL

# Comparison to Wilcoxon

In this artificial example, donor and batch are fictitious, so DESeq2's GLM $\beta$ estimates should not be that different from the Wilcoxon estimates. Here, we'll compare $\beta$s to auROC, which is essentially equivalent to the Wilxocon statistic. 

In [ ]:
## Wilcoxon on CP10K normalized counts 
exprs_norm <- singlecellmethods::normalizeData(pbmc3k@assays$RNA@counts, scaling_factor = 1e4, method = 'log')
dge_wilxocon <- wilcoxauc(exprs_norm, factor(pbmc3k@meta.data$seurat_annotations))


In [ ]:
head(dge_wilxocon)

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 8)
dplyr::inner_join(dge_wilxocon, res_mat, by = c('feature', 'group')) %>% 
    ggplot(aes(auc, stat)) + 
        geom_point(shape = '.') + 
        facet_wrap(~group, scales = 'free') + 
        geom_vline(xintercept = .5) + 
        geom_hline(yintercept = 0) + 
        labs(x = 'AUC', y = 'GLM beta') + 
        NULL

Most of the results agree, more or less. Interestingly, the Wilcoxon labels almost all genes as upregulated in DCs and CD16+ Monocytes and downregulated in Platelets. What's going on here? It turns out that DCs and CD16+ Monos are mRNA rich cells while platelets are mRNA poor cells overall. DESeq2 is able to account for this effect better than CP10K normalization. 


In [ ]:
options(repr.plot.height = 3, repr.plot.width = 5)
pbmc3k@meta.data %>% 
    subset(!is.na(seurat_annotations)) %>% 
    ggplot(aes(reorder(seurat_annotations, nCount_RNA), nCount_RNA)) + 
        geom_boxplot(outlier.shape = NA) + 
        geom_jitter(shape = '.', height = 0) + 
        coord_flip() + 
        labs(x = '') + 
        NULL

# Pairwise tests

Instead of 1-vs-all, let's do pairwise test and then summarize statistics conservatively. 

In [ ]:
data_collapsed <- collapse_counts(pbmc3k@assays$RNA@counts, 
                                  pbmc3k@meta.data, 
                                  c('seurat_annotations', 'donor', 'batch'))
head(data_collapsed$meta_data)

BUG: when testing all vs all pairwise, crashes

In [ ]:
# table(data_collapsed$meta_data$seurat_annotations)

In [ ]:
res_pair <- pseudobulk_deseq2(~seurat_annotations + donor + batch, 
                             data_collapsed$meta_data,
                             data_collapsed$counts_mat, verbose = TRUE, mode = 'pairwise')#, vals_test = c('B', 'NK'))


In [ ]:
res_min <- summarize_dge_pairs(res_pair, 'min')
# res_max <- summarize_dge_pairs(res_pair, 'max')

In [ ]:
## Compare to other modes

In [ ]:
dge <- Reduce(rbind, list(
    dplyr::mutate(dge_pairs_min, mode = 'Pairs_min'), 
    dplyr::mutate(dge_pairs_max, mode = 'Pairs_max'), 
    dplyr::mutate(dge_1va_col, mode = 'Onevall_collapse'), 
    dplyr::mutate(dge_1va_no, mode = 'Onevall_no')
    )) 

In [ ]:
plt_df <- dge %>% 
    dplyr::select(feature, log2FoldChange, mode) %>% 
    spread(mode, log2FoldChange)

plt_df <- plt_df[(rowSums(is.na(plt_df)) == 0), ]

In [ ]:
library(ggforce)
fig.size(6, 8)
plt_df %>% 
    ggplot(aes(x = .panel_x, y = .panel_y)) + 
        geom_point(shape = '.') + 
        geom_autodensity(alpha = 0.3, position = 'identity') + 
#         geom_autodensity(position = 'identity') + 
        facet_matrix(
            vars(Pairs_min, Pairs_max, Onevall_collapse, Onevall_no), 
            layer.diag = 2
        ) + 
        geom_vline(aes(xintercept = 0), linetype = 2) + 
        geom_hline(aes(yintercept = 0), linetype = 2) + 
        geom_abline(aes(slope = 1, intercept = 0)) + 
        NULL

In [ ]:
dge_pairs_min %>% 
    subset(stat > 0) %>% 
    dplyr::arrange(-stat) %>% 
    head()

In [ ]:
dge_pairs_max %>% 
    subset(stat > 0) %>% 
    dplyr::arrange(-stat) %>% 
    head()

# Test within clusters 

In [ ]:
if (!SeuratData::AvailableData()['ifnb.SeuratData', 'Installed']) {
    SeuratData::InstallData("ifnb")
}
data("ifnb")

## randomly assign donors
ifnb@meta.data$donor <- factor(sample(LETTERS[1:3], ncol(ifnb), TRUE))

In [ ]:
head(ifnb@meta.data)

In [ ]:
data_collapsed <- collapse_counts(ifnb@assays$RNA@counts, 
                                  ifnb@meta.data, 
                                  c('seurat_annotations', 'stim', 'donor'), 
                                     keep_n = TRUE)
head(data_collapsed$meta_data)

In [ ]:
data.table(data_collapsed$meta_data)[, sum(N), by = .(stim, seurat_annotations)] %>% 
    tidyr::spread(seurat_annotations, V1)

In [ ]:
res_mat <- pseudobulk_deseq2(~seurat_annotations + stim, 
                             data_collapsed$meta_data,
                             data_collapsed$counts_mat, 
                             verbose = TRUE, 
                             vals_test = c('pDC', 'B'),
                             mode = 'within')


In [ ]:
fig.size(2.5, 5)
data.table(res_mat)[, sum(padj < .01, na.rm = TRUE) / .N, by = group][order(-V1)] %>% 
    dplyr::inner_join(
        data.table(data_collapsed$meta_data)[, .(ncells = sum(N)), by = seurat_annotations],
        by = c('group' = 'seurat_annotations')
    ) %>% 
    ggplot(aes(100 * V1, ncells)) + 
        geom_point(shape = 21) + 
        scale_y_log10() + 
        geom_smooth(method = 'lm', se = FALSE) + 
        geom_label(aes(label = group), data = . %>% subset(V1 > .2)) + 
        NULL

In [ ]:
devtools::install_github('immunogenomics/presto')

In [ ]:
# devtools::document('..')

# Aggregate results faster 

This step should be super fast. The bottleneck is fitting models with lme4, not the rest of it. 


In [181]:
data_collapsed <- collapse_counts(pbmc3k@assays$RNA@counts, 
                                  pbmc3k@meta.data, 
                                  c('seurat_annotations', 'donor', 'batch'))
head(data_collapsed$meta_data)

CAREFUL: get_norm makes very strong assumptions about data



,seurat_annotations,donor,batch,N,logUMI
,<fct>,<fct>,<fct>,<int>,<dbl>
sample_28,Memory CD4 T,A,B,88,12.36016
sample_30,B,A,B,56,11.67743
sample_1,Memory CD4 T,A,A,83,12.35118
sample_29,CD14+ Mono,A,B,82,12.23536
sample_33,NK,A,B,33,11.09052
sample_10,Memory CD4 T,B,A,90,12.39591


In [185]:
formula=y~1+(1|seurat_annotations)+(1|batch)
design=pbmc3k@meta.data
response=pbmc3k@assays$RNA@counts
size_varname='logUMI'
features=sample(rownames(pbmc3k@assays$RNA@counts), 200)
# features=rownames(pbmc3k@assays$RNA@counts)
effects_cov=c('seurat_annotations')
ncore=20
nsim=1e3
family='poisson'
min_sigma=0
verbose=0L

In [186]:
library(lme4)

In [ ]:
    ## To make downstream things easier, give exposure variable a dedicated name 
    ## TODO: check that SIZE is valid exposure type variable 
    design$EXPOSURE <- design[[size_varname]]
    fstr <- gsub(size_varname, 'EXPOSURE', as.character(formula))
    formula <- as.formula(sprintf('%s~%s', fstr[[2]], fstr[[3]]), env=.GlobalEnv)

    if (verbose > 0) {
        message('Set up models')
    }
    
    ## fit an initial model just to get the names
    model_base <- fit_model.presto(formula, design, response[features[[1]], ], family)
    priornames_df <- as.data.frame(VarCorr(model_base))[, 1:3]
    if (isGLMM(model_base)) {
        ## glmer does not include residuals in VarCorr, lmer does
        priornames_df <- rbind(priornames_df, tibble(grp = 'Residual', var1 = NA, var2 = NA))
    }

    has_offset <- !all(map_lgl(model_base@resp$offset, identical, 0))
    betanames_df <- make_betanames_df(model_base, has_offset)
    
    ## set up parallel machinery 
    features <- intersect(features, rownames(response))
    if (ncore == 1) {
        future::plan(sequential)
    } else if (ncore %in% c(0, Inf)) {
        ncore <- availableCores()
        future::plan(multiprocess)
    } else {
        ## ncore weirdly not recognized by future
        .ncore <<- ncore
        future::plan(future::multiprocess(workers = .ncore))
        rm(.ncore)
    }
    
    if (verbose > 0) {
        message('Learn the models')
    }
    
    lres <- furrr::future_map(features, glmm_uni, formula, design, response, effects_cov, family, nsim, has_offset, min_sigma)
    names(lres) <- features
    lres <- lres[which(purrr::map_lgl(as.integer(map_int(lres, 'status')), identical, 0L))]



## Original

In [188]:
system.time({
    common_el <- purrr::reduce(map(lres, names), intersect) %>% setdiff('status')
    res <- map(common_el, function(name) {
        if (name == 'covmat') {
            purrr::reduce(purrr::map(lres, name), abind::abind, along = 3)
        } else {
            as.matrix(purrr::map_dfr(lres, name))            
        }
    })
    names(res) <- common_el
})


   user  system elapsed 
  0.116   0.002   0.117 

## Faster

In [130]:
sourceCpp('../src/utils.cpp')

covmat_list <- purrr::map(lres, 'covmat')

system.time({
    x <- purrr::reduce(covmat_list, abind::abind, along = 3)    
})

system.time({
    y <- collapse_mats(covmat_list, length(covmat_list))
    dnames <- dimnames(covmat_list[[1]])
    dnames[[3]] <- names(covmat_list)
    dimnames(y) <- dnames
})

max(abs(x - y))

   user  system elapsed 
  0.036   0.000   0.036 

   user  system elapsed 
  0.001   0.000   0.001 

[1] 0

In [164]:
sourceCpp('../src/utils.cpp')

beta_list <- purrr::map(lres, 'beta')
system.time({
    for (i in 1:10) {
        x <- as.matrix(purrr::map_dfr(lres, 'beta'))                
    }
})


system.time({
    for (i in 1:10) {
        y <- collapse_vecs(beta_list, length(beta_list))
        colnames(y) <- names(lres)        
    }
})

max(abs(x - y))

   user  system elapsed 
  0.024   0.000   0.024 

   user  system elapsed 
  0.007   0.000   0.006 

[1] 0

## Comparison

In [209]:
system.time({
    common_el <- purrr::reduce(map(lres, names), intersect) %>% setdiff('status')
    res <- map(common_el, function(name) {
        if (name == 'covmat') {
            purrr::reduce(purrr::map(lres, name), abind::abind, along = 3)
        } else {
            as.matrix(purrr::map_dfr(lres, name))            
        }
    })
    names(res) <- common_el
})



   user  system elapsed 
  0.108   0.004   0.112 

In [210]:
system.time({
    common_el <- purrr::reduce(map(lres, names), intersect) %>% setdiff('status')
    res0 <- map(common_el, function(name) {
        if (name == 'covmat') {
            covmat_list <- purrr::map(lres, 'covmat')
            res <- collapse_mats(covmat_list, length(covmat_list))
            dnames <- dimnames(covmat_list[[1]])
            dnames[[3]] <- names(covmat_list)
            dimnames(res) <- dnames
            return(res)          
        } else {
            vec_list <- purrr::map(lres, name)
            res <- collapse_vecs(vec_list, length(vec_list))
            colnames(res) <- names(lres)        
            return(res)
        }
    })
    names(res0) <- common_el
})



   user  system elapsed 
  0.025   0.000   0.025 

In [211]:
system.time({
    res <- list()
    res$beta <- purrr::map(lres, 'beta') %>% collapse_vecs(length(lres))
    colnames(res$beta) <- names(lres)

    res$epsilon <- purrr::map(lres, 'epsilon') %>% collapse_vecs(length(lres))
    colnames(res$epsilon) <- names(lres)

    res$epsilon_pearson <- purrr::map(lres, 'epsilon_pearson') %>% collapse_vecs(length(lres))
    colnames(res$epsilon_pearson) <- names(lres)

    res$prior_sd <- purrr::map(lres, 'prior_sd') %>% collapse_vecs(length(lres))
    colnames(res$prior_sd) <- names(lres)

    covmat_list <- purrr::map(lres, 'covmat')
    res$covmat <- collapse_mats(covmat_list, length(covmat_list))
    dnames <- dimnames(covmat_list[[1]])
    dnames[[3]] <- names(covmat_list)
    dimnames(res$covmat) <- dnames    
    
})


   user  system elapsed 
  0.009   0.000   0.009 

In [207]:
## Confirm same results 
map(common_el, function(name) {max(abs(res[[name]] - res0[[name]]))})

[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0